In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import ExcelWriter
 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
 
code_df.종목코드 = code_df.종목코드.map("{:06d}".format)
code_df = code_df[['종목코드', '회사명', '업종', '주요제품']]
code_df = code_df.rename(columns={'종목코드': 'code', '회사명':'name', '업종' : 'industry', '주요제품' : 'main_product'})

In [6]:
from bs4 import BeautifulSoup
import requests as re

In [2]:
code_df

,code,name,industry,main_product
0,000210,DL,기타 금융업,지주회사
1,004840,DRB동일,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매"
2,155660,DSR,1차 비철금속 제조업,합섬섬유로프
3,078930,GS,기타 금융업,지주회사/부동산 임대
4,001250,GS글로벌,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서..."
...,...,...,...,...
2486,393210,토마토시스템,소프트웨어 개발 및 공급업,UI솔루션
2487,217880,틸론,소프트웨어 개발 및 공급업,"Cloud Solution (D,A,E,Rstation)"
2488,222670,플럼라인생명과학,자연과학 및 공학 연구개발업,"강아지암치료제,돼지DNA테라피"
2489,331660,한국미라클피플사,기타 화학제품 제조업,"은나노스텝, 닥터오렌지 등"


In [17]:
def get_fnguide_table(code):
    
    try: 
        url = re.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%(code))
        url = url.content
        html = BeautifulSoup(url,'html.parser')
        body = html.find('body')
        fn_body = body.find('div',{'class':'fng_body asp_body'})
        ur_table = fn_body.find('div',{'id':'div15'})
        table = ur_table.find('div',{'id':'highlight_D_Y'})
        tbody = table.find('tbody')
        tr = tbody.find_all('tr')
        Table = DataFrame()
        print('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%(code))
        for i in tr:
            category = i.find('span',{'class':'txt_acd'})
            if category == None:
                category = i.find('th')   
                category = category.text.strip()


            value_list =[]
            
            j = i.find_all('td',{'class':'r'})
            
            for value in j:
                temp = value.text.replace(',','').strip()
                
                try:
                    temp = float(temp)
                    value_list.append(temp)
                except:
                    value_list.append(0)
            Table['%s'%(category)] = value_list
            
            thead = table.find('thead')
            tr_2 = thead.find('tr',{'class':'td_gapcolor2'}).find_all('th')
            year_list = []
            
            
            for i in tr_2:
                try:
                    temp_year = i.find('span',{'class':'txt_acd'}).text
                except:
                    temp_year = i.text
                year_list.append(temp_year)
            Table.index = year_list

            
        Table.reset_index(level=0, inplace=True)
        Table = Table.rename(columns={'index': 'year'}) 
        Table['code'] = code
        

        Table = Table.loc[Table.year.isin(['2014/12', '2015/12', '2016/12', '2017/12', '2018/12', '2019/12'])]
        return Table
    
    except: 
        print('error detection!')


In [18]:
get_fnguide_table('005930')

http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930


,year,매출액,영업이익,영업이익(발표기준),당기순이익,지배주주순이익,비지배주주순이익,자산총계,부채총계,자본총계,...,"<span class=""txt_acd"">ROA</span>","<span class=""txt_acd"">ROE</span>","<span class=""txt_acd"">EPS</span>","<span class=""txt_acd"">BPS</span>","<span class=""txt_acd"">DPS</span>","<span class=""txt_acd"">PER</span>","<span class=""txt_acd"">PBR</span>",발행주식수,"<span class=""txt_acd"">배당수익률</span>",code
0,2016/12,2018667.0,292407.0,292407.0,227261.0,224157.0,3104.0,2621743.0,692113.0,1929630.0,...,9.01,12.48,2735.0,24340.0,570.0,13.18,1.48,7033967.0,1.58,005930
1,2017/12,2395754.0,536450.0,536450.0,421867.0,413446.0,8422.0,3017521.0,872607.0,2144914.0,...,14.96,21.01,5421.0,28971.0,850.0,9.40,1.76,6454925.0,1.67,005930
2,2018/12,2437714.0,588867.0,588867.0,443449.0,438909.0,4540.0,3393572.0,916041.0,2477532.0,...,13.83,19.63,6024.0,35342.0,1416.0,6.42,1.10,5969783.0,3.66,005930
3,2019/12,2304009.0,277685.0,277685.0,217389.0,215051.0,2338.0,3525645.0,896841.0,2628804.0,...,6.28,8.69,3166.0,37528.0,1416.0,17.63,1.49,5969783.0,2.54,005930


In [ ]:
code_list = code_df['code'].values.tolist()
df = pd.DataFrame(columns = [])
 
for i in code_list:
    df = df.append(get_fnguide_table(i))